In [ ]:
import os
from glob import glob
import random
import time
import copy
import datetime as dt
import warnings
from collections import Counter
import itertools
import shutil
from pprint import pprint
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm.auto import tqdm
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import albumentations as A
import albumentations.pytorch
import sklearn
from sklearn.metrics import confusion_matrix
from joblib import parallel_backend
import ttach as tta
import timm
from timm.models.layers import Conv2dSame
from sklearn.metrics import f1_score
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.autograd import Variable
import torchvision.models as models
from focal_loss.focal_loss import FocalLoss
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':30,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정
time_now = dt.datetime.now()
run_id = time_now.strftime("%Y%m%d%H%M%S")

In [ ]:
all_img_list = glob.glob('./train/*/*')

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

In [ ]:
le = sklearn.preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'].values)

assert len(le.classes_) == 19

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            print(f'Best F1 score from now: {self.best_score}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
        
        return self.early_stop

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)
    
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.maxvit_t(pretrained=True)
        # self.backbone = timm.create_model('densenet201', pretrained=True, num_classes = 1000)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
def plot_confusion_matrix(
                        cm, classes, runid, epoch, 
                        f1, normalize=False, 
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(f'{title}-{runid}-{epoch}-{f1:.4f}')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f'./cms/cm-{runid}.jpg', dpi=400)
    plt.clf()

## Mixup & Cutmix

In [ ]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix_data(x, y, beta=1):
    lam = np.random.beta(beta, beta)
    rand_index = torch.randperm(x.size()[0]).cuda()
    target_a = y
    target_b = y[rand_index]              
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[rand_index, :, bbx1:bbx2, bby1:bby2]
    
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    
    return x, target_a, target_b, lam
                

## Transforms

In [ ]:
train_transform = A.Compose([
                            # A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            # A.CenterCrop(p=0.5, height=112, width=112),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(),
                            # A.RandomContrast(limit=0.2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
extra_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(),
                            A.RandomContrast(limit=0.2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

## 각 Fold 별 학습 진행

In [ ]:
# skf = sklearn.model_selection.StratifiedKFold(n_splits=20, shuffle=False)
skf = sklearn.model_selection.StratifiedKFold(n_splits=10, shuffle=False)
t = df.label

for fold, (train_index, test_index) in enumerate(skf.split(np.zeros(len(t)), t)):
    early_stopping = EarlyStopping(patience=10, verbose=True)
    data_train = df.loc[train_index]
    data_validation = df.loc[test_index]

    class_counts = data_train['label'].value_counts(sort=False).to_dict()
    num_samples = sum(class_counts.values())
    print(f'cls_cnts: {len(class_counts)}\nnum_samples:{num_samples}')
    labels = data_train['label'].to_list()

    # weight 제작, 전체 학습 데이터 수를 해당 클래스의 데이터 수로 나누어 줌
    class_weights = {l:round(num_samples/class_counts[l], 2) for l in class_counts.keys()}

    # class 별 weight를 전체 trainset에 대응시켜 sampler에 넣어줌
    weights = [class_weights[labels[i]] for i in range(int(num_samples))] 
    sampler = torch.utils.data.WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))
    
    # batch_size=16
    train_dataset = CustomDataset(df.loc[train_index]['img_path'].values, df.loc[train_index]['label'].values, train_transform)
    # extra_dataset = CustomDataset(df.loc[df.loc[train_index]['label'] == '9']['img_path'].values, df.loc[df.loc[train_index]['label'] == '9']['label'].values, extra_transform)
    validation_dataset = CustomDataset(df.loc[test_index]['img_path'].values, df.loc[test_index]['label'].values, test_transform)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=32,
        sampler=sampler,  # trainset에 sampler를 설정해줌
        shuffle=False,
        num_workers=0,
        pin_memory=True
        )
    validation_loader = DataLoader(
        validation_dataset, 
        batch_size=32,
        shuffle=False,
        num_workers=0)

    test = pd.read_csv('./test.csv')
    test_dataset = CustomDataset(test['img_path'].values, None, test_transform)

    test_loader = DataLoader(
        test_dataset, 
        batch_size=32,
        shuffle=False, 
        num_workers=0,
        pin_memory=True)

    dataloaders = {
        'train': train_loader,
        'val': validation_loader,
        'test': test_loader
    }

    dataset_sizes = {
        'train': len(train_dataset),
        'val': len(validation_dataset),
        'test': len(test_dataset)
    }

    # timm에서 모델을 가져옴
    device =  torch.device("cuda")
    # model = timm.create_model('tf_efficientnet_b7_ns', pretrained=True, num_classes=19)
    model = BaseModel()
    model.to(device)

    epochs = 300  # 보통 30~40 epoch에서 멈춥니다.
    # optimizer = optim.Adam(model.parameters(), lr=3e-4)
    optimizer = optim.AdamW(model.parameters(), lr=3e-4)
    # with weights 
    # The weights parameter is similar to the alpha value mentioned in the paper
    # weights_for_loss = torch.FloatTensor(list(class_weights.values())).to(device)
    # m = torch.nn.Softmax(dim=-1)
    # criterion = FocalLoss(gamma=2.0, weights=weights_for_loss)
    criterion = nn.CrossEntropyLoss()
    # scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    # scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-8)

    os.makedirs(f'./runs/{run_id}', exist_ok=True)
    os.makedirs(f'./cms/', exist_ok=True)
    
    since = time.time()
    best_f1 = 0.0
    scaler = torch.cuda.amp.GradScaler()

    fold_run_id = f'{run_id}_fold{str(fold)}'
 
    # 학습
    for epoch in range(epochs):
        print('-'*50)
        print(f'Fold: {fold}')
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        train_loss = 0.0

        for phase in ['train', 'val']:
            running_loss = 0.0
            cm_preds = []
            cm_labels = []
            model_preds = []
            model_labels = []

            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            for x, y in tqdm(iter(dataloaders[phase])):
                x = x.to(device)
                y = y.type(torch.LongTensor).to(device)
                optimizer.zero_grad()

                r, flag = np.random.rand(1), 1
                
                if phase == 'train': # CutMix
                    inputs, targets_a, targets_b, lam = cutmix_data(x, y) 
                    inputs, targets_a, targets_b = map(Variable, (inputs,
                                                            targets_a, targets_b))
                
                # if phase == 'train': # MixUp
                #     inputs, targets_a, targets_b, lam = mixup_data(x, y)
                #     inputs, targets_a, targets_b = map(Variable, (inputs,
                #                                             targets_a, targets_b))
                
                elif phase == 'val': flag = 0 # Original
                
                with torch.set_grad_enabled(phase == 'train'):
                    with torch.cuda.amp.autocast(enabled=True):
                        y_hat = model(x)
                        if flag == 0: loss = criterion(y_hat, y)
                        else: loss = mixup_criterion(criterion, y_hat, targets_a, targets_b, lam)
                    _, preds = torch.max(y_hat, 1)

                    if phase == 'train':
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()

                running_loss += loss.item() * x.size(0)
                
                model_labels += y.detach().cpu().numpy().tolist()
                model_preds += preds.detach().cpu().numpy().tolist()
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_f1 = f1_score(
                        model_labels, 
                        model_preds, 
                        average='weighted')
            print(f'[{phase}] Loss: {epoch_loss:.4f} Weighted F1: {epoch_f1:.4f}')
            
            if phase == 'val' and scheduler != None:
                scheduler.step(epoch_f1)

            # 체크포인트 저장
            if phase == 'val':
                if epoch_f1 > best_f1:
                    best_f1 = epoch_f1
                    torch.save(model, f'./runs/{run_id}/best_model_fold{fold}.pt')
                    confusion_mtx = confusion_matrix(model_labels, model_preds)
                    plot_confusion_matrix(confusion_mtx, classes=class_counts.keys(), runid=fold_run_id, epoch=epoch, f1=best_f1)
                else:
                    # torch.save(model, f'./runs/{run_id}/{epoch}-val_loss{epoch_loss}-val_f1{epoch_f1}.pt')
                    pass
            
        # EARLY STOPPING
        stop = early_stopping(epoch_f1)
        if stop:
            print("called")   
            break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val_F1: {:4f}'.format(best_f1))

    # 해당 fold의 checkpoint를 불러와 test
    device =  torch.device("cuda")
    checkpoint = f'./runs/{run_id}/best_model_fold{fold}.pt'
    print(f'CHECKPOINT LOADED: {checkpoint}')
    model = torch.load(checkpoint)
    model.to(device)
    model.eval()

    test_preds = []

    with torch.no_grad():
        for x in tqdm(iter(dataloaders['test'])):
                x = x.to(device)
                batch_pred = model(x)
                _, pred = torch.max(batch_pred, 1)
                pred = pred.detach().cpu().numpy().tolist()
                test_preds.extend(pred)

    # trainset에 fit_trainsform 되어있는 LabelEncoder로 inverse transform 해줌
    test_preds = le.inverse_transform(test_preds)

    submit = pd.read_csv('./sample_submission.csv')
    submit['label'] = test_preds
    
    submit.loc[submit['label'] == '0', 'label'] = '가구수정'
    submit.loc[submit['label'] == '1', 'label'] = '걸레받이수정'
    submit.loc[submit['label'] == '2', 'label'] = '곰팡이'
    submit.loc[submit['label'] == '3', 'label'] = '꼬임'
    submit.loc[submit['label'] == '4', 'label'] = '녹오염'
    submit.loc[submit['label'] == '5', 'label'] = '들뜸'
    submit.loc[submit['label'] == '6', 'label'] = '면불량'
    submit.loc[submit['label'] == '7', 'label'] = '몰딩수정'
    submit.loc[submit['label'] == '8', 'label'] = '반점'
    submit.loc[submit['label'] == '9', 'label'] = '석고수정'
    submit.loc[submit['label'] == '10', 'label'] = '오염'
    submit.loc[submit['label'] == '11', 'label'] = '오타공'
    submit.loc[submit['label'] == '12', 'label'] = '울음'
    submit.loc[submit['label'] == '13', 'label'] = '이음부불량'
    submit.loc[submit['label'] == '14', 'label'] = '창틀,문틀수정'
    submit.loc[submit['label'] == '15', 'label'] = '터짐'
    submit.loc[submit['label'] == '16', 'label'] = '틈새과다'
    submit.loc[submit['label'] == '17', 'label'] = '피스'
    submit.loc[submit['label'] == '18', 'label'] = '훼손'

    os.makedirs('./output/', exist_ok=True)
    submit.to_csv(f'./output/{run_id}_fold{fold}.csv', index=False)


## Folds 투표 

In [1]:
import pandas as pd
from collections import Counter
from glob import glob


# 경로 수정(run_id*.csv)
csvs = glob('./output/20230502015644*.csv')
# csvs2 = glob('./output/20221030183609*.csv')
# csvs.extend(csvs2)
print(len(csvs))

preds = []
for csv in csvs:
    f = pd.read_csv(csv)
    label = f['label'].tolist()
    preds.append(label)

# stacked = pd.read_csv('./stack/Stacked_3_Classes.csv')
# select = ['곰팡이', '석고수정', '오염']
out = []
cols = list(zip(*preds))
for i, c in enumerate(cols):
    most = Counter(c).most_common()[0][0]
    # if most in select: most = stacked.loc[i]['label']
    out.append(most)

print(out[:20])
ss = pd.read_csv('./sample_submission.csv')
ss['label'] = out
ss.to_csv('vote1234.csv', index=False)  # 구분 가능하게 경로 수정

10
['훼손', '오염', '오염', '몰딩수정', '오염', '오타공', '오염', '오염', '오염', '오타공', '몰딩수정', '오타공', '오염', '오염', '훼손', '훼손', '훼손', '걸레받이수정', '오염', '오염']
